## 노트북 내용

이 노트북은 watsonx.ai를 사용해서 소비 패턴 데이타를 분석해보는 유즈 케이스를 구현하고 있습니다.
이를 잘 이해하기 위해서는 Python 및 SQL에 대한 기본 지식이 필요하며 모든 코드는 Python 3.10 으로 작성되어 있습니다.


## 내용

이 노트북은 다음과 같은 단계로 구성되어 있습니다.

- [환경설정](#setup)
- [데이타베이스 설정](#database_setup)
- [프롬프트 템플릿 작성](#prompt_template)
- [사용자 질의 및 답변 생성](#execute_query)

<a id="setup"></a>
## 환경 설정

이노트북에 있는 샘플 코드를 실행하기 전에 다음 작업을 완료해야 합니다.

- 필요한 python package는 conda environment 혹은 python virtual environment에 python 3.10.12 기반의 독립적인 환경을 만든 후 pip install -r requirements_cp4d.txt를 사용해서 설치.
- Cloud Pak for Data 관리자에게 이 시스템에 접근할 수 있는 권한 정보를 획득.



In [ ]:
!pip install "langchain"
!pip install "langchain-community"
!pip install "langchain-experimental"
!pip install "ibm-watson-machine-learning==1.0.353"
!pip install "ibm_watsonx_ai==0.2.3"

In [ ]:
import os
import sqlite3
import csv
from datetime import datetime
import pytz
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.llms import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods


In [ ]:
# 데이터베이스 파일 삭제
!rm -f history.db

### Watson Machine Learning 연결 정보 확인

Cloud Pak for Data에서 제공하는 Watson Machine Learning 서비스에 접근하기 위한 인증 정보를 입력한다.
여기에는 `url`, `username` 그리고 `api_key`가 포함된다.

In [ ]:
username = 'jihunkim'
api_key = 'Your API Key'
url = 'https://cpd-watsonx.apps.elskr.zanity.net'
ver = '4.8'

wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": ver
}

### project id 확인
Foundation model에 접근하기 위한 Cloud Pak for Data의 project id를 확인 한다.
이 노트북이 Foundation model을 제공하는 Cloud Pak for Data이 Project 내에서 실행되는 경우 자동으로 환경 변수에서 가져올 수 있으나 Cloud Pak for Data 외부 환경에서 실행되는 경우는 연관된 project id를 확인한 후에 입력해 줘야 한다.

In [ ]:
try:
    project_id = os.environ["PROJECT_ID"]
    run_in_cp4d = True
except KeyError:
    project_id = "Your PROJECT_ID"
    run_in_cp4d = False

In [ ]:
if run_in_cp4d:

    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()
    wslib.download_file('spending-insight.csv')
    filename = 'spending-insight.csv'
else:
    filename = os.path.join(os.getcwd(), 'data', 'spending-insight.csv')

<a id="database_setup"></a>
## 데이타베이스 설정

신용카드 사용 내역을 저장할 데이타베이스를 생성한다. Lanchain은 SQLAlchemy 패키지 덕분에 MS SQL, MySQL, MariaDB, PostgreSQL 그리고 Oracle SQL등을 포함하는 여러 정류의 데이타 베이스에 연결할 수 있다.
여기서는 SQLite를 사용한다.


In [ ]:
# 데이터베이스 생성
db = SQLDatabase.from_uri("sqlite:///history.db")

# SQLite 데이터베이스에 연결하는 함수
def get_db_connection():
    conn = sqlite3.connect('history.db', check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    conn = get_db_connection()
    # 테이블 생성
    conn.execute("""
        CREATE TABLE IF NOT EXISTS transactions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            "date" DATE,
            "category" TEXT,
            "product" TEXT,
            "amount" INTEGER
        );
    """)
    
    # 테이블이 비어 있는 경우에 샘플 데이터를 입력한다.
    if conn.execute('SELECT COUNT(*) FROM transactions').fetchone()[0] == 0:
        with open(filename, 'r') as file:
            reader = csv.reader(file)
            next(reader)  # 헤더는 생략
            sample_data = list(reader)
            conn.executemany('INSERT INTO transactions ("date", "category", "product", "amount") VALUES (?, ?, ?, ?)', sample_data)
        
    # Commit the changes and close the connection
    conn.commit()
    conn.close()
init_db()

### 입력된 데이터 확인

모든 데이터가 잘 입력되어 있는 지 확인한다.

In [ ]:
conn = get_db_connection()
result = conn.execute('SELECT * from transactions')
for row in result.fetchall():
    print(row[:])

<a id="prompt_template"></a>
## 프롬프트 템플릿 작성

LLM에 질의하기 위한 프롬프트 템플릿을 작성한다.

### 테이블 칼럼 추출

LLM이 테이블에 어떤 데이터가 있는지 이해하도록 하기 위해 테이블의 칼럼을 추출한다.

In [ ]:
def get_table_columns(table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info({})".format(table_name))
    columns = cursor.fetchall()
    print(f"columns:{columns}")
    return [column[1] for column in columns]

table_name = 'transactions'
columns = get_table_columns(table_name)
display(columns)

<a id="models"></a>
## watsonx의 Foundation model 접근 설정

IBM watsonx의 foundation model들은 <a href="https://python.langchain.com/v0.2/docs/integrations/providers/ibm/#watsonxllm" target="_blank" rel="noopener no referrer">langchain에 의해 지원되는 LLM들의 목록</a>에 속한다.
여기서는 한글이 잘 동작한다고 평가되는 <a href="https://huggingface.co/mncai/llama2-13b-dpo-v7">mncai/llama2-13b-dpo-v7</a>를 사용한다.

WatsonxLLM class는 watsonx foundation model과 langchain 간의 interface를 제공해주는 class이다.

In [ ]:
model_id = 'mncai/llama2-13b-dpo-v7'
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1024,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}
watsonx_llama2_korean = WatsonxLLM(
    model_id=model_id,
    url=wml_credentials.get("url"),
    username=wml_credentials.get("username"),
    apikey=wml_credentials.get("apikey"),
    instance_id=wml_credentials.get("instance_id"),
    project_id=project_id,
    params=parameters
)

In [ ]:
# 데이터베이스 체인 생성
db_chain = SQLDatabaseChain.from_llm(watsonx_llama2_korean, db, verbose=True)

### 프롬프트 템플릿 생성

LLaMA용 템플릿에서는 시스템이 사용하는 메세지와 사용자의 메세지를 구분하기 위해 다음과 같은 tag를 사용한다.

    <<SYS>>/<</SYS>> : 시스템 (AI’s) 메세지.
    [INST]/[/INST] : 사용자 메세지.

In [ ]:
QUERY = """
    <<SYS>> 
    당신은 테스트를 SQL로 변환할 수 있는 매우 강력한 모델입니다. 당신의 역할은 데이터베이스에 대한 질문에 답하는 것입니다. 당신은 개인의 소비 성향을 나타내는 신용카드 사용내역을 저장하고 있는 테이블에 대해 질문을 받을 것입니다. 
    SQL쿼리 생성에 사용할 테이블 이름은 {table_name} 이며 칼럼들은 {columns} 입니다.
    당신은 답을 찾기 위해 SQLite 쿼리를 사용해야 합니다. 당신이 만들어서 사용할 쿼리문 내에 SQLite 문법에 어긋나는 것들, 예를 들면 DATE_TRUNC나 역따옴표(`) 또는 "```sql" 등을 포함해서는 안됩니다. 그런 다음 {table_name} 에 대해 쿼리를 실행하고 답을 생성하세요.
    가이드라인:
    - 위에 제시된 칼럼 이름을 정확히 사용한다.
    - 현재 time zone으로 결과를 필터링 한다: {time} 쿼리가 특정 날짜/시간동안의 기간을 명시하는 경우에만 사용. 날짜 필터링을 위해 ">=" or "<=" operators를 사용하거나 날짜를 월별로 묶기 위해 "GROUP BY strftime('%m', date)"을 사용한다. 데이터베이스의 날짜 형식은 'YYYY-MM-DD' 이다.
    - 쿼리 결과가 없는 경우에는 답변을 더블체크 하기 위해 SQLite 쿼리를 한 번 더 실행한다.  
    - 사용자 질문에서 '식료품', '식사', '일반생활비' 같은 특정 카테고리가 언급되는 경우 결과를 필터하기 위해 해당 카테고리에 "WHERE" 조건을 사용한다.
    - 특정 카테고리가 요청되지 않으면, 어떤 카테고리도 필터링 해서는 안된다.
    - 금액과 관련하여 '최대값'이나 '최소값'이 요청된 경우에는 SQL 함수 MAX() 나 MIN()을 사용하며 평균에는 AVG()를, 총 합계를 질문한 경우 SUM()을 사용한다.
    
    답변에는 생성한 SQL쿼리와 쿼리를 실행한 결과 그리고 사용자 질문에 대한 최종 답변을 포함합니다.
    <</SYS>>
    
    [INST] 
    사용자 질문 : {inquiry}
    답변: 
    [/INST]
    """

<a id="execute_query"></a>
## 사용자 질의 및 응답

사용자 질문을 입력하면 그에 맞는 쿼리를 생성하고 결과를 받은 다음에 최종 답변을 생성한다.

In [ ]:

inquiry = "2023년에 가장 자주 사용된 카테고리는 무엇입니까?"
# inquiry = "2023년 2사분기에 사용한 식료품에 사용한 금액은 모두 얼마입니까?"
# inquiry = "2023년 4사분기에 식사에 사용된 평균 금액은 얼마입니까?"


prompt = QUERY.format(table_name=table_name, columns=columns, time=datetime.now(pytz.timezone('Asia/Seoul')), inquiry=inquiry)
response = db_chain.run(prompt)


In [ ]:
print(response)